In [18]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
#read file
df = pd.read_csv('train.csv', header = None)
train_Y = pd.read_csv('trainLabels.csv', header = None)
#df.head()
print(df.info())
#train_Y.head()
train_Y = np.array(train_Y).ravel()


#scaling
SS = StandardScaler()
df_m1 = SS.fit_transform(df)
estimator = GradientBoostingClassifier()
print(cross_val_score(estimator, df_m1, train_Y, cv=5).mean())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
0     1000 non-null float64
1     1000 non-null float64
2     1000 non-null float64
3     1000 non-null float64
4     1000 non-null float64
5     1000 non-null float64
6     1000 non-null float64
7     1000 non-null float64
8     1000 non-null float64
9     1000 non-null float64
10    1000 non-null float64
11    1000 non-null float64
12    1000 non-null float64
13    1000 non-null float64
14    1000 non-null float64
15    1000 non-null float64
16    1000 non-null float64
17    1000 non-null float64
18    1000 non-null float64
19    1000 non-null float64
20    1000 non-null float64
21    1000 non-null float64
22    1000 non-null float64
23    1000 non-null float64
24    1000 non-null float64
25    1000 non-null float64
26    1000 non-null float64
27    1000 non-null float64
28    1000 non-null float64
29    1000 non-null float64
30    1000 non-null float64
31    1000 non-null float6

In [19]:
# Tuning Hypermeters
n_estimators = [50, 100, 200]
max_depth = [1, 3, 5, 7, 9]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(estimator, param_grid, scoring="accuracy", cv = 5, n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(df_m1, train_Y)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:   11.3s finished


In [20]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])
# 訓練模型
clf_bestparam.fit(df_m1, train_Y)

# 預測測試集
print(cross_val_score(clf_bestparam, df_m1, train_Y, cv=5).mean())

0.8779999999999999


In [28]:
x_test = pd.read_csv('test.csv', header = None)
y_pred = clf_bestparam.predict(x_test)
#print(y_pred)
submit = pd.DataFrame()
submit['ID'] = np.arange(1,9001)
submit['Solution'] = y_pred
submit.head()

#將預測結果儲存成CSV檔，上傳 Kaggle 成績。
submit.to_csv('C:\\Users\\jarla\\OneDrive\\桌面\\submit_HW48.csv', index = False, sep = ',', header = True)